## Plotting with Matplotlib

Although UXarray’s primary plotting API leverages the HoloViz ecosystem, users may prefer to compose custom figures using Matplotlib and Cartopy. UXarray provides standalone functions that integrate seamlessly into existing workflows, allowing you to visualize unstructured-grid data without reconstructing meshes or installing additional dependencies.

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

import uxarray as ux

In [ ]:
base_path = "../../test/meshfiles/ugrid/outCSne30/"
grid_path = base_path + "outCSne30.ug"
data_path = base_path + "outCSne30_vortex.nc"

uxds = ux.open_dataset(grid_path, data_path)

## Plotting Functions

UXarray’s standalone plotting functions live under the `uxarray.plot` namespace. For brevity, you can import them as:

In [ ]:
import uxarray.plot as uxplot

### Imshow

To quickly visualize data variables, UXarray provides the `uxarray.plot.imshow` function. It performs a screen-space sampling—mapping each Matplotlib pixel back to its corresponding unstructured-grid face, leveraging the optimized `Grid.get_faces_containing_point` method.

This produces an output image displaying the sampled data.

In [ ]:
uxplot.imshow(uxds["psi"])

## Integration with Cartopy and Matplotlib

UXarray’s Matplotlib functions are thin wrappers over Matplotlib and Cartopy, so you can:

- Use any Cartopy projection by passing in a configured GeoAxes
- Easily create subplots and insert plots
- Add map features like coastlines, borders and gridlines with standard Cartopy calls
- Define map bounds using `ax.set_extent` to focus on your region of interest

You get full Matplotlib + Cartopy functionality, no extra APIs to learn or boilerplate to write.

See the [Cartopy documentation](https://scitools.org.uk/cartopy/docs/latest/) for full details on supported functionality.


### Projections

UXarray’s Matplotlib helpers work hand‐in‐glove with Cartopy, so you can choose any map projection without extra setup:

- **Supply a Cartopy CRS**
  Pass your desired projection (e.g., PlateCarree, Robinson, Orthographic) directly into `uxplot.imshow`.
- **Automatic axis handling**
  The function creates or reuses a GeoAxes in that projection and resamples your data accordingly.
  -

A full list of supported projections can be found here.

Below, we render the `psi` field using the Robinson projection.


In [ ]:
uxplot.imshow(uxds["psi"], projection=ccrs.Robinson())

### Subplots and Figures

When building multi-panel figures, you can seamlessly integrate UXarray’s Matplotlib helpers into any existing workflow. If you provide a pre-configured GeoAxes (with your desired Cartopy projection), you don’t need to pass a projection argument into the `uxplot` functions.

Let’s create a basic side-by-side comparison of our data at different projections.


In [ ]:
fig = plt.figure(constrained_layout=True)
ax1 = fig.add_subplot(
    1, 2, 1, projection=ccrs.Orthographic(central_longitude=-10, central_latitude=35)
)
ax2 = fig.add_subplot(
    1, 2, 2, projection=ccrs.Orthographic(central_longitude=60, central_latitude=35)
)

# Render the same data on each axis with different center longitudes
uxplot.imshow(uxds["psi"], ax=ax1)
uxplot.imshow(uxds["psi"], ax=ax2)

# Set titles
ax1.set_title("Central Longitude = –10°")
ax2.set_title("Central Longitude = 60°")

plt.show()

### Map Features

UXarray’s Matplotlib helpers integrate seamlessly with Cartopy features. If you provide a pre-configured GeoAxes, simply call Cartopy feature methods directly, no extra parameters required.

In [ ]:
fig, ax = plt.subplots(
    subplot_kw={
        "projection": ccrs.Orthographic(central_longitude=-10, central_latitude=35)
    },
    constrained_layout=True,
)

uxplot.imshow(uxds["psi"], ax=ax)

# Add coastlines and international borders
ax.coastlines(resolution="110m", color="black", linewidth=1)
ax.add_feature(cfeature.BORDERS.with_scale("110m"), linestyle=":", linewidth=0.8)

plt.show()

### Setting Extents

Use Cartopy’s `ax.set_extent` to focus on your region of interest without slicing your data. Provide a list of coordinates in degrees (`[lon_min, lon_max, lat_min, lat_max]`) and UXarray’s plotting functions will honor the visible bounds.


In [ ]:
fig, ax = plt.subplots(
    subplot_kw={"projection": ccrs.PlateCarree()},
    constrained_layout=True,
)


ax.set_extent((-20, 20, -10, 10))

uxplot.imshow(uxds["psi"], ax=ax)


plt.show()